# JNMongo で複数ファイルの処理を行う
今回は前回の応用として、複数のファイルを連続的に処理する例を示します。    
今回利用させていただいたのは、
`resources/words.txt` に一覧されています。   
本来はもっと大規模に実験するべきですが、その場合は、IDR の ニコニコ大百科データセットを用いて下さい。

In [1]:
!cat resources/words.txt

エクス・アルビオ.json
コアラ.json
三枝明那.json
鈴原るる.json

## 処理をファイル名を引数にした関数に変える
前回は１ファイルについて処理をしていましたが、今回は複数ファイルを対象にしています。    
なので、これをファイルに対して不変の処理に処理に変える必要があります。

In [2]:
import pandas as pd
from parse import json_parser
from pathlib import Path
from typing import List, Dict
from service import utils, related_words, absts, contents

今回は

1. データを構造体の形式に持ち込む
2. その構造体についてMongoDBへ保存する

という手順で処理を行います。
Pythonで、構造体を定義する場合は辞書やクラスが考えられますが、今回はクラスを用います。

In [3]:
class ParsedArticle:
    def __init__(self, file: Path):
        self.raw_data = json_parser.read_json(file)
        self.title = utils.get_element(self.raw_data, "title")
        self.title_head = utils.get_element(self.raw_data, "title-head")
        self.keywords = utils.get_element(
            self.raw_data, "keywords", 
            message="this article has no keywords")       
        self.raw_related_words, self.related_words = related_words.extract_item(self.raw_data)
        self.related_words_len = len(self.related_words)
        self.abstruct, self.sections = self.parse_article()

    def parse_article(self):
        article = utils.get_element(
            self.raw_data, "article", message="this article has no body")
        import copy
        abstruct, _ ,left = absts.extract_abstruct(article)
        sections, _= contents.SectionParser().parse(left)
        return abstruct, sections

クラスが定義できたので、こちらを実験し、前回と同様の処理が行えていることを確認します。

In [4]:
sample_art = ParsedArticle(Path("./resources/三枝明那.json"))

In [5]:
def view_article(art: ParsedArticle):
    data = {}
    data["title"] = art.title
    data["title-head"] = art.title_head
    data["related-words"] = art.related_words
    data["related-words_raw"] = art.raw_related_words
    data["#related-words"] = art.related_words_len
    data["keywords"] = art.keywords
    data["abstruct"] = str(art.abstruct)
    data["section_keys"] = [str(sec.titles) for sec in art.sections]
    data["section_sample(key)"] = art.sections[0].titles
    data["section_sample(value)"] = art.sections[0].contents
    return pd.DataFrame(data.values(), index=data.keys(), columns=["value"])
view_article(sample_art)

,value
title,三枝明那
title-head,三枝明那とは (サエグサアキナとは) [単語記事] - ニコニコ大百科
related-words,"[⚖アンジュ・カトリーナ, 「にじさんじ」2019年上期加入ライバー, 📷瀬戸美夜子, 青道..."
related-words_raw,"[[{'content': ['にじさんじ', 'ベルモンド・バンデラス（尊敬するライバー）..."
#related-words,32
keywords,"[三枝明那, サエグサアキナ, にじさんじ, ショタコン, エクス・アルビオ, コラボ, デ..."
abstruct,[['三枝明那（さえぐさ あきな）とは、いちから株式会社が運営する「にじさんじ」所属のバーチ...
section_keys,"[('関連リンク',), ('関連項目',), ('脚注',)]"
section_sample(key),"(関連リンク,)"
section_sample(value),({'content': [{'content': ['三枝明那 – にじさんじ 公式サイト...


# 

他のファイルでも実行してみましょう。

In [6]:
sample_art = ParsedArticle(Path("./resources/エクス・アルビオ.json"))
view_article(sample_art)

,value
title,エクス・アルビオ
title-head,エクス・アルビオとは (エクスアルビオとは) [単語記事] - ニコニコ大百科
related-words,"[🌖夢月ロア, にじさんじ, 「にじさんじ」2019年上期加入ライバー, 🐺童田明治, ⚖️..."
related-words_raw,"[[{'content': ['バーチャルYouTuber', 'にじさんじ', 'いちから..."
#related-words,30
keywords,"[エクス・アルビオ, エクスアルビオ, にじさんじ, クズ, コラボ, バー, 英雄, ゲーム]"
abstruct,[['エクス・アルビオとは、いちから株式会社が運営する「にじさんじ」所属のバーチャルライバー...
section_keys,"[('関連リンク',), ('関連項目',)]"
section_sample(key),"(関連リンク,)"
section_sample(value),"({'content': [{'content': ['にじさんじ公式の紹介'], 'tag..."


In [7]:
sample_art = ParsedArticle(Path("./resources/コアラ.json"))
view_article(sample_art)

,value
title,コアラ
title-head,コアラとは (コアラとは) [単語記事] - ニコニコ大百科
related-words,"[コアラのマーチ, 多摩動物公園： 飼育・公開中（1頭のみ。2017年5月現在）, 動物の一..."
related-words_raw,"[[{'content': ['コアラのマーチ', 'オーストラリア', '動物', {'c..."
#related-words,8
keywords,"[コアラ, オーストラリア, 動物園, 動物, 名古屋市, 子供, マスコット]"
abstruct,"[['コアラとは、'], {'content': [{'content': ['オーストラリ..."
section_keys,"[('関連項目',), ('関連リンク',), ('脚注',)]"
section_sample(key),"(関連項目,)"
section_sample(value),"({'content': [{'content': ['コアラのマーチ'], 'tag': ..."


In [8]:
sample_art = ParsedArticle(Path("./resources/鈴原るる.json"))
view_article(sample_art)

,value
title,鈴原るる
title-head,鈴原るるとは (スズハラルルとは) [単語記事] - ニコニコ大百科
related-words,"[⚖アンジュ・カトリーナ, 🌖夢月ロア, にじさんじ, 「にじさんじ」2019年上期加入ライ..."
related-words_raw,"[[{'content': ['にじさんじ', 'いちから株式会社', 'バーチャルYouT..."
#related-words,23
keywords,"[鈴原るる, スズハラルル, コラボ, でびでび・でびる, にじさんじ, バーチャルライバー..."
abstruct,"[['（・w・🎀）'], ['鈴原るる（すずはら-）とは、いちから株式会社が運営する「にじさ..."
section_keys,"[('実況配信したゲーム', 'レトロゲーム'), ('実況配信したゲーム', 'ホラーゲー..."
section_sample(key),"(実況配信したゲーム, レトロゲーム)"
section_sample(value),"({'content': [{'content': ['超魔界村'], 'tag': 'li..."


## 各データを保存する
複数のファイルで正しく処理が出来ることを確認したので、それぞれを保存していきます。   
この項目についても効率化を目指して、全てのファイルに対して不変となるような一連の手続きを作ります。

In [9]:
from boundary.absts import Abst
from boundary.contents import Content
from boundary.dockeywords import DocKeywords
from boundary.keywords import Keyword
from boundary.related_words import Related_Words
from service.contents import SectionTree

from pymongo import MongoClient
from bson.objectid import ObjectId
from getpass import getpass

from typing import List, Union, Tuple

In [10]:
class Boundary:
    def __init__(self, 
            username: str="root", password: str ="passwd", 
            host: str="127.0.0.1", port: int=27017):
        client = MongoClient(host=host, 
            port=port, username=username, password=password)
        self.abst_db = Abst(client)
        self.content_db = Content(client)
        self.keyword_db = Keyword(client)
        self.dockeywords_db = DocKeywords(client)
        self.relatedwords_db = Related_Words(client)

    def reset(self):
        self.abst_db.reset()
        self.content_db.reset()
        self.keyword_db.reset()
        self.dockeywords_db.reset()
        self.relatedwords_db.reset()

    def insert_keyword(
        self, keyword: Union[str, List[str], Tuple[str]]):
        if type(keyword) is str:
            self.keyword_db.insert(keyword)
        elif type(keyword) in [list, tuple]:
            [self.keyword_db.insert(k) for k in keyword]
        else:
            raise NotImplementedError()

    def get_keyword(self, keyword: Union[str, List[str]]):
        if type(keyword) is str:
            return self.keyword_db.find_object(keyword)
        elif type(keyword) is list:
            return [self.keyword_db.find_object(k) 
                    for k in keyword]
        else:
            raise NotImplementedError()

    def recover_keyword(
        self, 
        keyword_id: Union[ObjectId, List[ObjectId]]):
        if type(keyword) is ObjectId:
            return self.keyword_db.get_by_id(keyword_id)
        elif type(keyword) is list:
            return [self.keywrod_db.get_by_id(i) for i in keyword]
        else:
            raise NotImplementedError()
    
    def insert_dockeywords(
        self, title: Union[str, ObjectId],
        keywords: Union[List[str], List[ObjectId]]):
        if type(title) is str:
            title = self.get_keyword(title)
        if len(keywords) != 0 and type(keywords[0]) is str:
            keywords = self.get_keyword(keywords)
        self.dockeywords_db.insert(title, keywords)

    def insert_relatedwords(
        self, title: Union[str, ObjectId],
        related_words: Union[List[str], List[ObjectId]],
        raw_related_words: Dict):
        if type(title) is str:
            title = self.get_keyword(title)
        if len(related_words) != 0 and type(related_words[0]) is str:
            related_words = self.get_keyword(related_words)
        self.relatedwords_db.insert(
            title,
            related_words,
            raw_related_words)
    def insert_abstruct(
        self, title: Union[str, ObjectId], abstruct: Dict):
        if type(title) is str:
            title = self.get_keyword(title)
        self.abst_db.insert(title, abstruct)

    def insert_sections(
        self, title: Union[str, ObjectId], 
        sections: List[SectionTree]):
        if type(title) is str:
            title = self.get_keyword(title)
        for sec in sections:
            self.content_db.insert(
                title,
                sec.titles,
                sec.contents)
    
    def insert_article(
        self, article: ParsedArticle):
        self.insert_keyword(article.title)
        self.insert_keyword(article.related_words)
        self.insert_keyword(article.keywords)
        for sec in article.sections:
            self.insert_keyword(sec.titles)
        self.insert_dockeywords(
            article.title,
            article.keywords)
        self.insert_relatedwords(
            article.title,
            article.related_words,
            article.raw_related_words)
        self.insert_abstruct(
            article.title,
            article.abstruct)
        self.insert_sections(
            article.title,
            article.sections)

    @property
    def info(self):
        return {
            "abst": len(self.abst_db),
            "content": len(self.content_db),
            "keyword": len(self.keyword_db),
            "dockeywords": len(self.dockeywords_db),
            "relatedwords": len(self.relatedwords_db)}
    def __repr__(self):
        return str(self.info)
    

In [11]:
username="root"
password=getpass()

In [12]:
bound = Boundary(username=username, password=password)
# 初期化
bound.reset()

# ステータスの表示
print(bound)
pd.DataFrame(bound.info.values(), index=bound.info.keys(), columns=["value"])

{'abst': 0, 'content': 0, 'keyword': 0, 'dockeywords': 0, 'relatedwords': 0}


,value
abst,0
content,0
keyword,0
dockeywords,0
relatedwords,0


サンプルに１ファイルを適用してみます。    
JNMongo_startup.ipynb と同様の結果が得られることが確認できます。

In [13]:
sample_art = ParsedArticle(Path("./resources/三枝明那.json"))
bound.insert_article(sample_art)
pd.DataFrame(bound.info.values(), index=bound.info.keys(), columns=["value"])

,value
abst,1
content,3
keyword,42
dockeywords,1
relatedwords,1


それでは複数のファイルにこれを適用してみます。

In [14]:
bound.reset()
parent = Path("./resources")
for path in parent.glob("*.json"):
    art = ParsedArticle(path)
    bound.insert_article(art)
pd.DataFrame(bound.info.values(), index=bound.info.keys(), columns=["value"]) 

,value
abst,4
content,14
keyword,79
dockeywords,4
relatedwords,4


以上の手続きを用いることで、大量のファイルについても同様に処理できると思います。    
次は、 visualize.ipynb でデータ同士の繋がりを観察してみます。